In [168]:
def get_target(y_data) :
    
    y_values = []
    
    for item in y_data:
        
        if item not in y_values:
            y_values.append(item)
            
    return y_values

In [169]:
def getFrequency(y_data, label) :
    
    length = len(y_data)
    target = get_target(y_data)
    frequency = 0 
    matching = 0
        
    for item in y_data :
            
        if label == item :
                
            matching=matching+1
                
    frequency = matching/length
    
    return frequency           

In [170]:
import math

def getEntropy(y_data) :
    
    entropy = 0
    target = get_target(y_data)
    
    for tg in target :
        
        f = getFrequency(y_data,tg)
        entropy = entropy - f*math.log(f,2)
        
    return entropy

In [171]:
import numpy as np

def getProbability(column) :
    
    length = len(column)
    target = get_target(column)
    probability = []
    
    for tg in target :
        matching = 0
        for item in column :
            
            if tg == item :
                matching=matching+1
                
            prob = matching/length
        probability.append(prob)
    
    return probability

In [172]:
def getConditionalEntropy(column,feature_value,y_data) :
        
        new_Y = []
        conditionalEntropy=0
        a=0
        
        for item in column :
                        
            if feature_value == item :
                
                new_Y.append(y_data[a])

            a=a+1 
                
        probability = np.array(getProbability(new_Y)) 
        
        for p in probability :
    
            conditionalEntropy = conditionalEntropy-p*math.log(p,2)
        
        return conditionalEntropy   

In [173]:
def getInformationGain(column,y_data):
    
    probabilities = getProbability(column)
    entropies = []
    values=get_target(column)
    informationGain=0
    conditionalEntropy=0
    
    for item in values:
        
        x=getConditionalEntropy(column,item,y_data)
        entropies.append(x)
        
    a=0
    for el in probabilities:
        
        conditionalEntropy = conditionalEntropy + (probabilities[a]*entropies[a])
        a=a+1
        
    informationGain = getEntropy(y_data) - conditionalEntropy
    
    return informationGain

In [204]:
def computeBestSplit(x_data,y_data,feature_names):
    
    ig=[]
    a=0
    for column in x_data.T:
        ig.append(getInformationGain(column,y_data))
        a=a+1
        
        
    return feature_names[np.argmax(ig)]

In [205]:
from sklearn import datasets
import numpy as np

X_data= np.array ([['Sunny','Hot','High','Weak'],
        ['Sunny','Hot','High','Strong'],
        ['Overcast','Hot','High','Weak'],
        ['Rain','Mild','High','Weak'],
        ['Rain','Cool','Normal','Weak'],
        ['Rain','Cool','Normal','Strong'],
        ['Overcast','Cool','Normal','Strong'],
        ['Sunny','Mild','High','Weak'],
        ['Sunny','Cool','Normal','Weak'],
        ['Rain','Mild','Normal','Weak'],
        ['Sunny','Mild','Normal','Strong'],
        ['Overcast','Mild','High','Strong'],
        ['Overcast','Hot','Normal','Weak'],
        ['Rain','Mild','High','Strong']])

y_data=np.array(['no','no','yes','yes','yes','no','yes','no','yes','yes','yes','yes','yes','no'])

feature_names=np.array(['outlook','temperature','humidity','wind'])

computeBestSplit(X_data,y_data,feature_names)

'outlook'